# Modifier les données

**Vous pouvez bien sûr exécuter les commandes via l'invite de commande ou VS Code si vous le souhaitez**

Exemples pour : 

- Création de table avec `CREATE`
- Insertion de données `INSERT`
- Mise à jour de données `UPDATE`
- Suppression de données `DELETE`

### Setup Python / Environnement

- Installer le package `dotenv`

```bash 
pip3 install python-dotenv
``` 

In [4]:
# Point d'exclamation pour exécuter depuis le notebook
!pip3 install python-dotenv

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


```bash
# contenu du fichier .env (qui se trouve à la racine du repository)
POSTGRESQL_LOCAL_USER=selim
POSTGRESQL_LOCAL_PASSWORD=selim
``` 

In [2]:
import os
from dotenv import load_dotenv
# Ajouter aux variables environnementales celles contenues dans le .env
load_dotenv("../../../.env")

USER = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")


In [8]:
import psycopg2
from psycopg2 import sql

try:
    # Connexion à PostgreSQL
    conn = psycopg2.connect(user=USER, password=PASSWORD, host="localhost", port="5432")
    conn.autocommit = True
    cur = conn.cursor()

    # Pour pouvoir ré exécuter le code
    cur.execute("DROP DATABASE IF EXISTS example_db;")

    # Création d'une nouvelle base de données
    cur.execute("CREATE DATABASE example_db;")
    print("Database created successfully.")

    # Connexion à la nouvelle base de données
    conn.close()
    conn = psycopg2.connect(
        dbname="example_db", 
        user=USER, 
        password=PASSWORD, 
        host="localhost", 
        port="5432"
    )
    cur = conn.cursor()

    # Création d'une table
    cur.execute("""
        CREATE TABLE IF NOT EXISTS products (
            product_name VARCHAR(255) NOT NULL,
            unit_price DECIMAL NOT NULL,
            quantity INT NOT NULL
        );
    """)
    print("Table created successfully.")
    
    
    cur.execute("""SELECT * FROM products;""")
    print(len(list(cur.fetchall())))

    # Insertion de données
    cur.execute("""
        INSERT INTO products (product_name, unit_price, quantity) 
        VALUES 
            ('Apple', 0.50, 100),
            ('Banana', 0.30, 150);
    """)
    print("Data inserted successfully.")

    cur.execute("""SELECT * FROM products;""")
    print(len(list(cur.fetchall())))
    
    # Insertion de plusieurs lignes
    cur.executemany("""
        INSERT INTO products (product_name, unit_price, quantity) 
        VALUES (%s, %s, %s);
    """, [('Orange', 0.40, 200), ('Pear', 0.60, 120)])
    print("Multiple data inserted successfully.")

    # Mise à jour des données
    cur.execute("""
        UPDATE products 
        SET unit_price = 0.55 
        WHERE product_name = 'Apple';
    """)
    print("Data updated successfully.")

    # Mise à jour avec JOIN (en supposant qu'une autre table existe)
    # Exemple : UPDATE products SET ... FROM another_table WHERE products.id = another_table.product_id;

    # Suppression des données
    cur.execute("""
        DELETE FROM products 
        WHERE product_name = 'Banana';
    """)
    print("Data deleted successfully.")


    # Fermeture de la connexion
    cur.close()
    conn.close()

except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL database: {e}")

Database created successfully.
Table created successfully.
0
Data inserted successfully.
2
Multiple data inserted successfully.
Data updated successfully.
Data deleted successfully.


# Transactions
## Principe
- Une transaction est une unité de travail qui comprend une ou plusieurs opérations
- Les propriétés ACID (Atomicité, Cohérence, Isolation, Durabilité) garantissent l'intégrité des transactions.


## Syntaxe
- `BEGIN` : Commence une transaction.
- `COMMIT` : Valide les modifications effectuées pendant la transaction.
- `ROLLBACK` : Annule toutes les modifications non validées dans la transaction courante.

**Avantages** : 
- Sécurité : Assure l'intégrité des données.
- Consistance : Maintient la base de données dans un état cohérent.
- Isolation : Protège les transactions des interférences mutuelles.

**Inconvénients**
- Complexité : Gestion supplémentaire dans le code.
- Performance : Peut réduire la performance sur de grosses transactions.


In [3]:
import psycopg2

try:
    # Connexion à la base de données PostgreSQL
    conn = psycopg2.connect(user=USER, password=PASSWORD, host="localhost", port="5432")
    cur = conn.cursor()

    # Création d'une table 'accounts'
    cur.execute("""
        DROP TABLE IF EXISTS accounts;
        CREATE TABLE accounts (
            nom VARCHAR(100) NOT NULL,
            valeur DEC(15,2) NOT NULL
        );
    """)

    # Début d'une transaction
    cur.execute("BEGIN;")

    # Insertion d'un enregistrement dans la table 'accounts'
    cur.execute("INSERT INTO accounts(nom, valeur) VALUES('Hubert', 10000);")

    # Mise à jour des données
    cur.execute("UPDATE accounts SET valeur = valeur - 1500 WHERE nom = 'Hubert';")

    # Annulation des changements
    cur.execute("ROLLBACK;")

    # Validation des changements
    cur.execute("COMMIT;")

    # Fermeture de la connexion
    cur.close()
    conn.close()
    
except psycopg2.Error as e:
    print(f"Erreur de connexion à la base de données PostgreSQL : {e}")